네이버 지도는 명칭/상호로 Geocode가 동작되지 않는다. 주소로만 작동.
해결방안: 
1. 구글의 Geocode를 활용하여 좌표를 구함
2. 얻어진 좌표로 네이버의 Direction 5를 활용하여 경로 찾기

In [1]:
import googlemaps

gmaps = googlemaps.Client(key={GOOGLE_API_KEY})

In [10]:
origin = input("출발지를 입력하세요: ")
destination = input("도착지를 입력하세요: ")

geocode_origin = gmaps.geocode((origin), language='ko') # 한국어 설정으로 결과값을 받아온다.
geocode_destination = gmaps.geocode((destination), language='ko')

latitude_origin  = geocode_origin[0]["geometry"]["location"]["lat"] # 리스트에서 위도 추출
longitude_origin = geocode_origin[0]["geometry"]["location"]["lng"] # 리스트에서 경도 추출
latitude_destination  = geocode_destination[0]["geometry"]["location"]["lat"] # 리스트에서 위도 추출
longitude_destination = geocode_destination[0]["geometry"]["location"]["lng"] # 리스트에서 경도 추출

print(f"출발지 {origin}의 위도는 {latitude_origin} 입니다.")
print(f"출발지 {origin}의 경도는 {longitude_origin} 입니다.")
print(f"도착지 {destination}의 위도는 {latitude_destination} 입니다.")
print(f"도착지 {destination}의 경도는 {longitude_destination} 입니다.")

출발지 서울시청의 위도는 37.566535 입니다.
출발지 서울시청의 경도는 126.9779692 입니다.
도착지 스파크플러스 홍대입구의 위도는 37.5577182 입니다.
도착지 스파크플러스 홍대입구의 경도는 126.9253904 입니다.


In [11]:
origin = (longitude_origin, latitude_origin)
destination = (longitude_destination, latitude_destination)
print(origin)
print(destination)

(126.9779692, 37.566535)
(126.9253904, 37.5577182)


In [16]:
import json, urllib
from urllib.request import Request, urlopen

# *-- Directions 5 활용 코드 --*
option = 'traoptimal'   # 최적 경로
# option : 탐색옵션 [최대 3개, traoptimal(기본 옵션) 
# / trafast, tracomfort, traavoidtoll, traavoidcaronly]

def get_optimal_route(start, goal) :
    client_id = {NAVER_API_KEY_ID}
    client_secret = {NAVER_API_KEY}
    # start=/goal=/(waypoint=)/(option=) 순으로 request parameter 지정
    url = f"https://naveropenapi.apigw.ntruss.com/map-direction-15/v1/driving?&start={start[0]},{start[1]}&goal={goal[0]},{goal[1]}&option={option}"
    request = urllib.request.Request(url)
    request.add_header('X-NCP-APIGW-API-KEY-ID', client_id)
    request.add_header('X-NCP-APIGW-API-KEY', client_secret)
    
    response = urllib.request.urlopen(request)
    res = response.getcode()
    
    if (res == 200) :
        response_body = response.read().decode('utf-8')
        return json.loads(response_body)
            
    else :
        print('ERROR')
        
results = get_optimal_route(origin, destination)

In [18]:
# *-- 목적지까지의 종합 정보 추출(총 거리:meters, 총 소요시간: milliseconds, 요금 등) --*
message = {
    'message' : results['message'], # traoptimal은 option 에 따라 다르게 줄 것
    'option' : list(results['route'].keys())[0],
    'total_distance' : results['route']['traoptimal'][0]['summary']['distance'],
    'total_duration' : results['route']['traoptimal'][0]['summary']['duration'],
    'fares' : {'toll' : results['route']['traoptimal'][0]['summary']['tollFare'],
              'taxi' : results['route']['traoptimal'][0]['summary']['taxiFare'],
              'fuel' : results['route']['traoptimal'][0]['summary']['fuelPrice']}
}
print(message)
print('-------------------------')

# *-- 목적지까지의 guidence와 각각의 거리, 소요시간 정보 추출 --*
temp = [ (guide['instructions'], guide['distance'] , guide['duration'] / 1000 ) \
for guide in results['route']['traoptimal'][0]['guide'] ]

print(temp)

{'message': '길찾기를 성공하였습니다.', 'option': 'traoptimal', 'total_distance': 6912, 'total_duration': 1670515, 'fares': {'toll': 0, 'taxi': 10200, 'fuel': 952}}
-------------------------
[("'세종대로20길' 방면으로 좌회전", 48, 28.106), ("'세종대로' 방면으로 우회전", 33, 20.7), ('세종대로사거리에서 유턴', 253, 50.424), ("시청에서 '신촌로터리' 방면으로 우회전", 546, 116.753), ("서소문고가에서 '신촌로터리, 이대입구' 방면으로 왼쪽 고가차도 진입", 509, 109.467), ("충정로사거리에서 '양화대교, 마포대교' 방면으로 좌회전", 977, 146.617), ("아현교차로에서 '양화대교, 신촌로터리' 방면으로 우회전", 291, 41.71), ("창전삼거리에서 '홍익대' 방면으로 우회전", 2495, 668.785), ("'와우산로' 방면으로 좌회전", 372, 124.046), ("홍익대앞에서 '홍대입구역' 방면으로 우회전", 583, 143.202), ("홍대입구역에서 '동교동삼거리' 방면으로 우회전", 353, 143.532), ('우회전', 412, 63.666), ('목적지', 40, 13.507)]


In [24]:
distance_km = message['total_distance']/1000    # m -> km 변환
duration_minute = round(message['total_duration']/60000)    # miliseconds -> min 변환

print(f"차량 이동 시 총 거리는 {distance_km}km, 시간은 {duration_minute}분 입니다.")

차량 이동 시 총 거리는 6.912km, 시간은 28분 입니다.


#### 참고: NAVER reverse geocode

In [ ]:
import requests
import json

# NCP 콘솔에서 복사한 클라이언트ID와 클라이언트Secret 값
client_id = {NAVER_API_KEY_ID}
client_secret = {NAVER_API_KEY}

# 헤더
headers = {
    "X-NCP-APIGW-API-KEY-ID": client_id,
    "X-NCP-APIGW-API-KEY": client_secret,
}

# 좌표 (경도 + 위도)
coords_origin = str(longitude_origin)+","+str(latitude_origin)
# coords_destination = str(latitude_destination)+","+str(longitude_destination)
output = "json"
orders = 'addr'
endpoint = "https://naveropenapi.apigw.ntruss.com/map-reversegeocode/v2/gc"
url_origin = f"{endpoint}?coords={coords_origin}&output={output}&orders={orders}"

# 요청
res = requests.get(url_origin, headers=headers)
res.json()

{'status': {'code': 0, 'name': 'ok', 'message': 'done'},
 'results': [{'name': 'addr',
   'code': {'id': '1144012100', 'type': 'L', 'mappingId': '09440121'},
   'region': {'area0': {'name': 'kr',
     'coords': {'center': {'crs': '', 'x': 0.0, 'y': 0.0}}},
    'area1': {'name': '서울특별시',
     'coords': {'center': {'crs': 'EPSG:4326',
       'x': 126.9783882,
       'y': 37.5666103}},
     'alias': '서울'},
    'area2': {'name': '마포구',
     'coords': {'center': {'crs': 'EPSG:4326',
       'x': 126.901491,
       'y': 37.5663245}}},
    'area3': {'name': '동교동',
     'coords': {'center': {'crs': 'EPSG:4326',
       'x': 126.9251193,
       'y': 37.557576}}},
    'area4': {'name': '',
     'coords': {'center': {'crs': '', 'x': 0.0, 'y': 0.0}}}},
   'land': {'type': '1',
    'number1': '167',
    'number2': '24',
    'addition0': {'type': '', 'value': ''},
    'addition1': {'type': '', 'value': ''},
    'addition2': {'type': '', 'value': ''},
    'addition3': {'type': '', 'value': ''},
    'ad